# 2024-02-01 34 - Is nontuned resilience better than nontuned nonresilience.ipynb

## Parameters

In [36]:
#FIGURES_FOLDER="../figures/icml"

## Libraries and setup

In [145]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [147]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, Latex

In [148]:
from utils import wandb_utils

## Data Download

In [8]:
df=pd.read_pickle("../data_backups/2024_02_01_1038am__icml_backup_data.pkl")
# For now, only resilience

resilience_df = df.query('constraint_type=="constant_resilience"')
non_resilence_df = df.query('constraint_type=="constant"')
erm_df = df.query('constraint_type=="erm"')
df = pd.concat([resilience_df, non_resilence_df, erm_df])

In [7]:
resilience_df.shape, non_resilence_df.shape, erm_df.shape

((288144, 108), (282960, 108), (48384, 108))

## Data processing
This section contains processing specific to this experiment set. If you clone this notebook. Remove or refactor this section.

### Label constraint_levels (`constraint_idx` column)
useful for joining when the constraint values are different for each model.

In [13]:
df.query('constraint_type=="constant_resilience"').groupby(['data_path','model','pred_len'])['constraint_level'].nunique().reset_index()

,data_path,model,pred_len,constraint_level
0,electricity.csv,Autoformer,96,6
1,electricity.csv,Autoformer,192,6
2,electricity.csv,Autoformer,336,6
3,electricity.csv,Autoformer,720,6
4,electricity.csv,Informer,96,6
5,electricity.csv,Informer,192,6
6,electricity.csv,Informer,336,6
7,electricity.csv,Informer,720,6
8,electricity.csv,Reformer,96,6
9,electricity.csv,Reformer,192,5


In [72]:
def process_erm_vs_constrained(df,constraint_type):
    """Takes a DF that has one set of constrained runs (multiple constraint_levels) and a set of ERM runs, adds constraint_idx and broadcassts
    constraint data to ERM for plotting
    
    NO GRIDSEARCH.
    """
    # There are three constraint levels per run, add a rank column, sorted by constraint level.
    df=df.copy()#df_constrained.copy()
    #display(df[['seed','data_path','model','pred_len','constraint_level']].drop_duplicates().sort_values(['data_path','model','pred_len','constraint_level']).head())
    df.loc[df.constraint_type==constraint_type,'constraint_idx'] = df.query('constraint_type==@constraint_type').groupby(['data_path','model','pred_len'])['constraint_level'].rank(method='dense', ascending=True).astype(int)
    #df.loc[df.constraint_type=='erm','constraint_idx'] = 0
    #df['constraint_idx'] = df['constraint_idx'].astype(int)
    ranked_constraint_levels_summary = df[['run_id','data_path','model','pred_len','constraint_type','constraint_level','constraint_idx']].drop_duplicates('run_id').sort_values(['data_path','model','pred_len','constraint_level'])
    
    df_constrained = df.query('constraint_type==@constraint_type').copy()
    
    #TODO UNCOMMENTED THIS FOR THIS PARTICULAR NOTEBOOK (N34)
    #df_constrained,best_runs = gridsearch(df_constrained,objective_col='mse', gridsearch_keys=['model', 'pred_len', 'data_path'],reference_cols=['run_id'],split='val')
    df_erm = df.query('constraint_type=="erm"').copy()
    print(df_erm.constraint_type.unique())
    
    #(data_path,model,pred_len,constraint_level,constrained_unconstraled)
    distinct_constraints = df_constrained[['data_path','model','pred_len','constraint_idx','constraint_level']].drop_duplicates()
    
    # crossjoin the ERM mse to all constraint levels
    df_erm_w_broadcasted_constraint_levels = df_erm.drop(columns=['constraint_idx','constraint_level']).merge(distinct_constraints,on=['data_path','model','pred_len'],how='left')
    if df_erm_w_broadcasted_constraint_levels.constraint_idx.isna().any():
        print("WARNING! Found Nas in constraint_idx after merge, this means some runs are missing")
    df_to_plot = pd.concat([df_constrained,df_erm_w_broadcasted_constraint_levels],axis=0)

    # sanity check
    display(df_erm_w_broadcasted_constraint_levels.shape)
    print(f"Three constraints per experiment, broadcasted dataset size be equal to {df_erm.shape[0]*6}")
    display(df_to_plot.shape)
    print(f"Final dataset size should be equal to {df_constrained.shape[0]+df_erm.shape[0]*6}")
    
    return df_to_plot, ranked_constraint_levels_summary
    
#Counts dont match because might be missing a couple of runs.
resilience_broadcasted=process_erm_vs_constrained(df.query('constraint_type=="constant_resilience" or constraint_type=="erm"'),"constant_resilience")[0]
nonresilience_broadcasted=process_erm_vs_constrained(df.query('constraint_type=="constant" or constraint_type=="erm"'),"constant")[0]
df_to_plot = pd.concat([resilience_broadcasted,nonresilience_broadcasted],axis=0)

['erm']


(285408, 109)

Three constraints per experiment, broadcasted dataset size be equal to 290304


(573552, 109)

Final dataset size should be equal to 578448
['erm']


(282960, 109)

Three constraints per experiment, broadcasted dataset size be equal to 290304


(565920, 109)

Final dataset size should be equal to 573264


In [75]:
#looks good
df_to_plot.query('constraint_type=="erm"')[['data_path','model','pred_len','constraint_type','constraint_level','constraint_idx','split']].drop_duplicates()\
        .sort_values(['data_path','model','pred_len','constraint_type','constraint_level','constraint_idx']).head(40)

,data_path,model,pred_len,constraint_type,constraint_level,constraint_idx,split
189075,electricity.csv,Autoformer,96,erm,0.111,1.0,train
189651,electricity.csv,Autoformer,96,erm,0.111,1.0,test
190227,electricity.csv,Autoformer,96,erm,0.111,1.0,val
191237,electricity.csv,Autoformer,96,erm,0.113,1.0,train
191813,electricity.csv,Autoformer,96,erm,0.113,1.0,test
192389,electricity.csv,Autoformer,96,erm,0.113,1.0,val
189076,electricity.csv,Autoformer,96,erm,0.115,2.0,train
189652,electricity.csv,Autoformer,96,erm,0.115,2.0,test
190228,electricity.csv,Autoformer,96,erm,0.115,2.0,val
191236,electricity.csv,Autoformer,96,erm,0.117,2.0,train


### Add constraint violation metric

In [76]:
# Adding the constraint violation column
df_to_plot["constraint_violation"] = (df_to_plot.mse-df_to_plot.constraint_level).clip(lower=0)
df_to_plot['is_infeasible'] = (df_to_plot.constraint_violation>0)*1

# sanity check
dd=df_to_plot[['run_id','model','type','step','mse','constraint_level','constraint_violation','is_infeasible']] 
print("For ERM")
display(dd.query('type=="ERM"').query('constraint_violation>0').head())
print("For Constrained")
display(dd.query('type=="Constrained"').query('constraint_violation>0').head())


For ERM


,run_id,model,type,step,mse,constraint_level,constraint_violation,is_infeasible
2,k6rl9exh,Transformer,ERM,0,0.095688,0.051,0.044688,1
3,k6rl9exh,Transformer,ERM,0,0.095688,0.048,0.047688,1
4,k6rl9exh,Transformer,ERM,0,0.095688,0.049,0.046688,1
8,k6rl9exh,Transformer,ERM,1,0.060131,0.051,0.009131,1
9,k6rl9exh,Transformer,ERM,1,0.060131,0.048,0.012131,1


For Constrained


,run_id,model,type,step,mse,constraint_level,constraint_violation,is_infeasible
171080,onjq1549,Transformer,Constrained,296,3.227262,3.198,0.029262,1
171093,onjq1549,Transformer,Constrained,309,3.201611,3.198,0.003611,1
171094,onjq1549,Transformer,Constrained,310,3.245584,3.198,0.047584,1
171095,onjq1549,Transformer,Constrained,311,3.205746,3.198,0.007746,1
171118,onjq1549,Transformer,Constrained,334,3.208134,3.198,0.010134,1


### Summary metrics per run
Summarize all three runs into metrics per split by averaging mse and constraint levels.

In [154]:
# Calculate mean and std for mse and constraint_violation. Rename them mean_{metric} and std_{metric

summary_metrics_per_run = df_to_plot.groupby([
    "data_path",
    "model",
    "pred_len",
    "run_id",
    "Algorithm", #Harmless carryover col for legacy purposes (verified the counts are the same)
    'type',
    "constraint_type",
    "constraint_idx",
    "split",
    ])[
        'mse','constraint_violation','infeasible_rate','run_id','constraint_level',
        'pct_50_total_test',
        'pct_95_total_test',
        'pct_99_total_test',
        'pointwise/cvar/001',
        'pointwise/std'
        
        ].aggregate(
        mse=('mse','mean'),
        # Risk and spread
        window_std=('mse','std'),
        pct_50_total_test=('pct_50_total_test','mean'),
        pct_95_total_test=('pct_95_total_test','mean'),
        pct_99_total_test=('pct_99_total_test','mean'),
        pointwise_cvar_001=('pointwise/cvar/001','mean'),
        pointwise_std=('pointwise/std','mean'),
        
        #std_mse=('mse','std'),
        mcv=('constraint_violation','mean'),
        #std_mcv=('constraint_violation','std'),
        mean_infeasible_rate=('infeasible_rate','mean'),
        constraint_level=('constraint_level','mean'),
        #std_infeasible_rate=('infeasible_rate','std'),
        run_count=('run_id','nunique')
        ).reset_index().round(5)
summary_metrics = summary_metrics_per_run.groupby([
    "data_path",
    "model",
    "pred_len",
    'type', #ERM or constrained
    "split",
    "Algorithm",
    "constraint_type",
    "constraint_idx",
    ]).aggregate(
        mse=('mse','mean'),
        
        window_std=('window_std','mean'),
        pct_50_total_test=('pct_50_total_test','mean'),
        pct_95_total_test=('pct_95_total_test','mean'),
        pct_99_total_test=('pct_99_total_test','mean'),
        pointwise_cvar_001=('pointwise_cvar_001','mean'),
        pointwise_std=('pointwise_std','mean'),
        
        std_mse=('mse','std'),
        mcv=('mcv','mean'),
        std_mcv=('mcv','std'),
        mean_infeasible_rate=('mean_infeasible_rate','mean'),
        std_infeasible_rate=('mean_infeasible_rate','std'),
        run_count=('run_count','mean'),
        constraint_level=('constraint_level','mean'),
    ).reset_index()
display(summary_metrics_per_run.shape)
display(summary_metrics.shape)
summary_metrics.head()

/home/jporras/miniconda3/envs/autoformer/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


(2568, 20)

(2562, 22)

,data_path,model,pred_len,type,split,Algorithm,constraint_type,constraint_idx,mse,window_std,pct_50_total_test,pct_95_total_test,pct_99_total_test,pointwise_cvar_001,pointwise_std,std_mse,mcv,std_mcv,mean_infeasible_rate,std_infeasible_rate,run_count,constraint_level
0,electricity.csv,Autoformer,96,Constrained,test,electricity Autoformer,constant,1.0,0.20018,0.01513,0.16475,0.37851,0.64604,NaN,NaN,NaN,0.08718,NaN,1.00000,NaN,1.0,0.113
1,electricity.csv,Autoformer,96,Constrained,test,electricity Autoformer,constant,2.0,0.20433,0.01493,0.16652,0.39240,0.67780,NaN,NaN,NaN,0.08733,NaN,1.00000,NaN,1.0,0.117
2,electricity.csv,Autoformer,96,Constrained,test,electricity Autoformer,constant,3.0,0.20447,0.01488,0.16649,0.39330,0.67727,NaN,NaN,NaN,0.08447,NaN,1.00000,NaN,1.0,0.120
3,electricity.csv,Autoformer,96,Constrained,test,electricity Autoformer,constant,4.0,0.20195,0.01544,0.16543,0.38701,0.66791,NaN,NaN,NaN,0.05095,NaN,0.98551,NaN,1.0,0.151
4,electricity.csv,Autoformer,96,Constrained,test,electricity Autoformer,constant,5.0,0.20733,0.01548,0.16733,0.41407,0.71087,NaN,NaN,NaN,0.04733,NaN,0.95458,NaN,1.0,0.160


### == **END OF PREPROCESSING** ==

---
---

## Plots

### ERM vs. Constrained on 50th percentile (one seed)
TO DO: Reproduce the ERM vs constrained per model for each window size. Try to unify into one plot if possible.

In [ ]:
# PARAMS
split="test"
#constraint_idx = 1 # use the 50 percentile constraint to plot 1=25,2=50,3=75
seed=2021

# Filter
df_pct50_and_erm = df_to_plot[(df_to_plot.seed==seed) & (df_to_plot.split==split)].copy()

PRED_LENS=sorted(df_to_plot.pred_len.unique().tolist())
#PRED_LENS=[96]
for pred_len in PRED_LENS:
    # Add data for constant line
    df_current_plot = df_pct50_and_erm.query('pred_len==@pred_len').copy()
    #constraint_level = df_current_plot.constraint_level.unique()[0]
    #df_current_plot['constraint_level'] = constraint_level
    display(Markdown(f"#### **ERM Vs Constrained {pred_len} (split={split})**"))

    g = sns.FacetGrid(df_current_plot, col='model', row='data_path', hue='type', height=3, aspect=1.5, legend_out=True)
    #g.map(sns.lineplot, "step", "constraint_level", alpha=.7)
    g.map(sns.scatterplot, "step", "mse", alpha=.7)
    g.add_legend()
    #TODO fix and uncomment
    #g.savefig(f"{FIGURES_FOLDER}/constrained_vs_erm_{pred_len}_{str(constraint_level).replace('.','')}.png")
    plt.show()

### (old) ERM vs constrained

In [ ]:
from IPython.display import display, Markdown
to_plot = df_to_plot[(df_to_plot.split=='test')].copy()
for (data_path,pred_len), series in to_plot.groupby(['data_path',"pred_len"]):
    constraints = series.constraint_idx.unique()
    print(constraints)
    constraints = constraints[constraints!=-1] # to iterate over all constraints explored.
    for constraint_idx in constraints:
        display(Markdown(f"### Dataset: {data_path} pred_len: {pred_len} constraint_idx: {constraint_idx}"))
        plot_series = series#series[series["constraint_idx"]==constraint_idx]
        
        #means_per_experiment = plot_series.groupby(['pred_len','constraint_level','Algorithm','model','epoch','type'])['mse'].mean().reset_index()
        #display(means_per_experiment)
        # do a pivot where model are the rows, and the columns constrained and unconstrained
        #display(means_per_experiment.pivot(index=['model'], columns='type', values='mse').reset_index())

        # sns.scatterplot(x="step", y="mse", hue="Algorithm",style="split", data=plot_series)
        
        # plt.axhline(y=constraint_level, color='r', linestyle='-')
        # plt.title(f"Pred Length: {pred_len}, Constraint_level: {constraint_level}")
        # plt.show()

        # This same scatterplot but in a grid by algorithm. 
        g = sns.FacetGrid(plot_series, col="model", hue="type")
        g.map(sns.scatterplot, "step", "mse", alpha=.7)
        g.add_legend()
        # add the hline
        # for ax in g.axes:
        #     ax.axhline(y=constraint_level, color='r', linestyle='-')
        plt.show()
        # export this plot to latex. Remove the dot in constraint_level decimal
        #TODO fix
        #g.savefig(f"{FIGURES_FOLDER}/constrained_vs_erm_{data_path.replace('.','')}_{pred_len}_{str(constraint_level).replace('.','')}.png")
        # Fix [Errno 2] No such file or directory: 'figures/electricity.csv_96_0.17_individual.png'


In [38]:
# sample Autoformer 192 weather.csv constant_resilience
df_to_plot.query('data_path=="weather.csv" and model=="Autoformer" and pred_len==192 and constraint_type=="constant_resilience"').drop_duplicates(["data_path","model","pred_len","constraint_level","constraint_idx"])[[
    'data_path','type','model','pred_len','constraint_type','constraint_level','constraint_idx']].sort_values('constraint_idx')


,data_path,type,model,pred_len,constraint_type,constraint_level,constraint_idx
280224,weather.csv,ERM,Autoformer,192,constant_resilience,0.689,0
280225,weather.csv,ERM,Autoformer,192,constant_resilience,0.559,0
280226,weather.csv,ERM,Autoformer,192,constant_resilience,0.696,0
280227,weather.csv,ERM,Autoformer,192,constant_resilience,0.383,0
280228,weather.csv,ERM,Autoformer,192,constant_resilience,0.464,0
280229,weather.csv,ERM,Autoformer,192,constant_resilience,0.580,0
383760,weather.csv,Constrained,Autoformer,192,constant_resilience,0.383,1
478944,weather.csv,Constrained,Autoformer,192,constant_resilience,0.464,2
375696,weather.csv,Constrained,Autoformer,192,constant_resilience,0.559,3
480816,weather.csv,Constrained,Autoformer,192,constant_resilience,0.580,4


### More random explorations
Ideas: 
1. Loss over time
2. Infeasibility rate over time
3. Loss shape across epochs (see how it evolves)
4. Plotting dual variables

## Result tables
1. Reproduce the pivot table from notebook 11, perhaps add standard deviation reports.
2. Explore and find other interesting table visualziation

In [177]:
df_to_plot[df_to_plot['pointwise/std'].isna()][['experiment_tag']].drop_duplicates().values

array([['e19_icml_resilience_val'],
       ['e19_icml_resilience_train'],
       ['e16_finaljan_allerm_replicated'],
       ['e18_icml_constant_constrained_loose'],
       ['e17_finaljan_constant_constrained_replicated']], dtype=object)

### Pivot tables (summary)

In [226]:
def style_pivot_table_best_per_params(pivoted_df):
    pivoted_df=pivoted_df.copy()
    pivoted_df.columns.names = ['metric','constraint_type','model']
    mask = pd.DataFrame(False, index=pivoted_df.index, columns=pivoted_df.columns)
    for keys,subset in  pivoted_df.groupby(level=['metric','model'],axis=1):
        mask.loc[:,subset.columns] = subset.eq(subset.min(axis=1), axis=0)
        
    ##STYLE
    # truncate to 4 decimals for display
    pivoted_df=pivoted_df.style.format("{:.3f}")
    pivoted_df=pivoted_df.apply(lambda s: ['font-weight: bold;text-decoration: underline' if v else '' for v in mask.loc[s.name]], axis=1)
    return pivoted_df

def data_path_renaming(x):
    if x=="exchange_rate.csv":
        x='Exchange'
    x=x.replace(".csv","")
    if x=="electricity":
        x="ECL"
    else: 
        x=x.replace(".csv","").replace("_"," ").title()
    return latex_scaffolding_dataset(x)
def latex_scaffolding_dataset(x):
    # Add rotatebox
    x=r'\rotatebox'+'{90}{'+x+'}'
    x=r'\multirow{4}{*}{'+x+'}'
    return x
def text_bf(x):
    return r'\textbf{'+str(x)+'}'

def latex_style_best(pivoted_df):
    pivoted_df=pivoted_df.copy()
    pivoted_df.columns.names = ['metric','constraint_type','model']
    mask = pd.DataFrame(False, index=pivoted_df.index, columns=pivoted_df.columns)
    for keys,subset in  pivoted_df.groupby(level=['metric','model'],axis=1):
        mask.loc[:,subset.columns] = subset.eq(subset.min(axis=1), axis=0)
    #if mask is true, add text_bf
    boldface_df = pivoted_df.copy().applymap(lambda x: f"{x:.3f}")
    boldface_df[mask] = boldface_df.applymap(text_bf)
    return boldface_df

pd.set_option('display.max_columns', None)
def compute_pivot_over_metric(summary_metrics,pivot_value_columns=['window_std'],split='test'):
    pivoted_tables=[]
    data_paths=summary_metrics.data_path.unique()
    for data_path in data_paths:
        tp2=summary_metrics.query(f'data_path==@data_path and split==@split').copy()

        #GROUPING TEH VALS
        # grouped_values=tp2.groupby(['pred_len','model','constraint_type'])[[
        #     'mse',
        #     'mcv',
        #     'std_mse',
        #     'std_mcv',
        #     'mean_infeasible_rate',
        #     'std_infeasible_rate',
        #     'window_std'
        #     ]].mean().reset_index()
        grouped_values=tp2.groupby(['pred_len','model','constraint_type'])[pivot_value_columns].mean().reset_index()
        
        # Pivot of models versus constrained and ERM, values are mse and mse_diff
        all_results = grouped_values
        
        # Pivot all results, sort ascending by pred_len, then mean_violation on constrained
        pivoted=all_results.pivot(
            index=['pred_len'],
            columns=['constraint_type','model'], 
            values=pivot_value_columns
        )
        
        pivoted=style_pivot_table_best_per_params(pivoted)
        #display(pivoted)
        pivoted_tables.append(pivoted)
    result_table = pd.concat([d.data for d in pivoted_tables],keys=data_paths,names=['data_path'])

    return result_table

constraint_type="constant"
constraint_idx=2
split='test'

summary_metrics_to_plot = summary_metrics.query('(constraint_type==@constraint_type or constraint_type=="erm" or constraint_type=="constant_resilience") and constraint_idx==@constraint_idx').copy()
#disable erm

#summary_metrics_to_plot = summary_metrics_to_plot.query('constraint_type!="erm"')
summary_metrics_to_plot['constraint_type'] = summary_metrics_to_plot['constraint_type'].replace({'constant':'1Ours','constant_resilience':'2Ours+R','erm':'ERM'})
# sort values so that Ours and Ours(R) are always first
#summary_metrics_to_plot['constraint_type2'] = pd.Categorical(summary_metrics_to_plot['constraint_type'], ['1Ours', '2Ours+R', '3ERM'])
#summary_metrics_to_plot=summary_metrics_to_plot.sort_values('constraint_type2')

display(Markdown(f"# Pivot tables ({constraint_type}) for split={split} idx {constraint_idx}"))
for metric in [
    #'mse',
    'window_std',
    # 'pointwise_std',
    # 'mcv',
    # 'mean_infeasible_rate',
    # 'pct_50_total_test',
    # 'pct_95_total_test',
    # 'pct_99_total_test',
    # 'pointwise_cvar_001',
    
    ]:
    result_table = compute_pivot_over_metric(summary_metrics_to_plot,pivot_value_columns=[metric],split='test')
    display(Markdown(f"## {metric}"))
    # add bold
    display(style_pivot_table_best_per_params(result_table))
    #latex styling
    latex_table=result_table.copy()

    latex_table.index=latex_table.index \
        .set_levels([data_path_renaming(level) for level in latex_table.index.levels[0]],level=0)

    #TODO FIX the styling function to add textbf
    latex_table=latex_style_best(latex_table)
    # drop the metric level in the multiindex
    

    latex_table.columns=latex_table.columns.droplevel(0)
    # Swap the column multiindex to model on top
    latex_table=latex_table.swaplevel(0,1,axis=1)

    # Show only :.4f
    #latex_table=latex_table.applymap(lambda x: f"{x:.4f}")
    # Rename data_path to None
    latex_table=latex_table.rename_axis(index={'data_path':None,'pred_len':''},columns={'model':'','constraint_type':''})
    #Reorganize the column multiindex so that constraint_type Ours is first
    
    
    print(latex_table.to_latex(escape=False,multicolumn_format='c',multicolumn=True))


# Pivot tables (constant) for split=test idx 2

## window_std

\begin{tabular}{llllllllllllll}
\toprule
                                         &     & \multicolumn{3}{c}{Autoformer} & \multicolumn{3}{c}{Informer} & \multicolumn{3}{c}{Reformer} & \multicolumn{3}{c}{Transformer} \\
                                         &     &           1Ours &         2Ours+R &    ERM &           1Ours &         2Ours+R &    ERM &           1Ours &         2Ours+R &             ERM &           1Ours &         2Ours+R &             ERM \\
{} & {} &                 &                 &        &                 &                 &        &                 &                 &                 &                 &                 &                 \\
\midrule
\multirow{4}{*}{\rotatebox{90}{ECL}} & 96  &           0.015 &  \textbf{0.013} &  0.015 &  \textbf{0.013} &           0.018 &  0.014 &           0.011 &  \textbf{0.008} &           0.011 &  \textbf{0.007} &           0.008 &           0.007 \\
                                         & 192 &           0.022 &  \t

### Comparing variance and CoV ERM vs constrained

In [42]:
# Calculate mean and std for mse and constraint_violation. Rename them mean_{metric} and std_{metric
#ToDO filter one seeed and split
stepwise_metrics = df_to_plot[df_to_plot.constraint_idx.isin([0,2])].query("split=='test'").groupby([
    "run_id",
    "Algorithm", #Harmless carryover col for legacy purposes (verified the counts are the same)
    "model",'type',
    "constraint_level",
    "constraint_idx",
    "pred_len",
    "data_path",
    "split",
    'step'#STEPWISE
    ])['mse','constraint_violation','infeasible_rate','run_id'].aggregate(
        mse=('mse','mean'),
        #std_mse=('mse','std'),
        mcv=('constraint_violation','mean'),
        #std_mcv=('constraint_violation','std'),
        mean_infeasible_rate=('infeasible_rate','mean'),
        #std_infeasible_rate=('infeasible_rate','std'),
        run_count=('run_id','nunique')
        ).reset_index().round(5)
summary_stepwise=stepwise_metrics.groupby([
    "run_id",
    "Algorithm", #Harmless carryover col for legacy purposes (verified the counts are the same)
    "model",
    'type',
    "constraint_level",
    "constraint_idx",
    "pred_len",
    "data_path",
    "split"]).aggregate(
        mse=('mse','mean'),
        std_mse=('mse','std'),
        mcv=('mcv','mean'),
        std_mcv=('mcv','std'),
        mean_infeasible_rate=('mean_infeasible_rate','mean'),
        std_infeasible_rate=('mean_infeasible_rate','std'),
        run_count=('run_count','mean'),
        step_count=('step','count')
    ).reset_index()
summary_stepwise

/home/jporras/miniconda3/envs/autoformer/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  del sys.path[0]


,run_id,Algorithm,model,type,constraint_level,constraint_idx,pred_len,data_path,split,mse,std_mse,mcv,std_mcv,mean_infeasible_rate,std_infeasible_rate,run_count,step_count
0,092u33lj,AllModels_electricity.csv Reformer,Reformer,ERM,-1.000,0,96,electricity.csv,test,0.298882,0.010611,1.298882,0.010611,1.0,0.0,1.0,96
1,0roswz2a,AllModels_electricity.csv Autoformer,Autoformer,ERM,-1.000,0,336,electricity.csv,test,0.334163,0.043029,1.334163,0.043029,1.0,0.0,1.0,336
2,0wvmszxw,AllModels_electricity.csv Reformer,Reformer,ERM,-1.000,0,720,electricity.csv,test,0.316104,0.007018,1.316104,0.007018,1.0,0.0,1.0,720
3,2ldijjak,AllModels_weather.csv Transformer,Transformer,ERM,-1.000,0,720,weather.csv,test,1.002190,0.289485,2.002190,0.289485,1.0,0.0,1.0,720
4,2p7rxqys,AllModels_weather.csv Informer,Informer,ERM,-1.000,0,96,weather.csv,test,0.435266,0.118225,1.435266,0.118225,1.0,0.0,1.0,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,x03d8gyt,AllModels_electricity.csv Transformer,Transformer,ERM,-1.000,0,336,electricity.csv,test,0.280047,0.019536,1.280047,0.019536,1.0,0.0,1.0,336
58,yhsan1ax,AllModels_weather.csv Informer,Informer,ERM,-1.000,0,192,weather.csv,test,0.514648,0.158067,1.514648,0.158067,1.0,0.0,1.0,192
59,yl7kgu6w,exchange_rate Reformer,Reformer,Constrained,0.189,2,192,exchange_rate.csv,test,1.402268,0.148336,1.213268,0.148336,1.0,0.0,1.0,192
60,zbxw8wgb,electricity Reformer,Reformer,Constrained,0.177,2,720,electricity.csv,test,0.316280,0.007203,0.139280,0.007203,1.0,0.0,1.0,720


In [43]:
summary_stepwise.groupby(['run_id','type','constraint_idx',"Algorithm","model"]).size().reset_index()

,run_id,type,constraint_idx,Algorithm,model,0
0,092u33lj,ERM,0,AllModels_electricity.csv Reformer,Reformer,1
1,0roswz2a,ERM,0,AllModels_electricity.csv Autoformer,Autoformer,1
2,0wvmszxw,ERM,0,AllModels_electricity.csv Reformer,Reformer,1
3,2ldijjak,ERM,0,AllModels_weather.csv Transformer,Transformer,1
4,2p7rxqys,ERM,0,AllModels_weather.csv Informer,Informer,1
...,...,...,...,...,...,...
57,x03d8gyt,ERM,0,AllModels_electricity.csv Transformer,Transformer,1
58,yhsan1ax,ERM,0,AllModels_weather.csv Informer,Informer,1
59,yl7kgu6w,Constrained,2,exchange_rate Reformer,Reformer,1
60,zbxw8wgb,Constrained,2,electricity Reformer,Reformer,1


In [44]:
dd_weather = summary_stepwise.query('data_path=="weather.csv" and pred_len==96').copy()
dd_weather.groupby(['run_id','constraint_idx',"Algorithm","model","constraint_idx"])
#dd_weather.pivot(index=['run_id','constraint_idx',"Algorithm","model","constraint_idx"], columns='type', values='mse')

#### Pivot tables stepwise

In [45]:
#Note, variance is nan because there is only one run per parameter set
weather = summary_stepwise.query('data_path=="weather.csv"').copy()
electricity = summary_stepwise.query('data_path=="electricity.csv"').copy()
for name,dataset in [('weather',weather), ('electricity',electricity)]:
    display(Markdown(f"# {name}"))
    tp2=dataset.copy(deep=True)
    #tp2=weather.query('split=="test"').copy()
    # Compute 'infeasible amount' as the difference between the mse and the constraint_level, 0 if mse < constraint
    
    # Split into constrained and ERM
    tp2_constrained=tp2.query('type=="Constrained"').copy()
    tp2_erm_baseline=tp2.query('type=="ERM"').copy()

    distinct_constraints = tp2_constrained[['pred_len','model','constraint_level']].drop_duplicates()

    #GROUPING TEH VALS
    grouped_values=tp2.groupby(['pred_len','constraint_level','constraint_idx','model','type'])[[
        'mse','mcv',
        'std_mse','std_mcv',
        'mean_infeasible_rate','std_infeasible_rate']].mean().reset_index()

    # Pivot of models versus constrained and ERM, values are mse and mse_diff
    all_results = grouped_values#pd.concat([grouped_constrained,grouped_erm_baseline_broadcasted],axis=0)
    # Pivot all results, sort ascending by pred_len, then mean_violation on constrained

    pivoted=all_results.pivot(
        #index=['pred_len','constraint_idx'], 
        index=['pred_len'], 
        columns=['type','model'],
        values=['mse','std_mse','mcv','std_mcv','mean_infeasible_rate','std_infeasible_rate']
    ).query('constraint_level>0').round(4)
    simplified = all_results.pivot(
        index=['pred_len','constraint_level'],
        columns=['type','model'],
        values=['mse','mcv']
    )
    pd.set_option('display.max_columns', None)
    print("SIMPLIFIED 192 MSE")
    # show all pred lengths (index)
    for pred_len in [96, 192, 336, 720]:    
        print(simplified.index.get_level_values(0).unique())
        simp_mse_192=simplified[simplified.index.get_level_values(0)==pred_len][['mse']]
        sim_mcv_192=simplified[simplified.index.get_level_values(0)==pred_len][['mcv']]

        # Flatten the multiindex
        def prettify(d):
            d = d.copy()
            d.columns = d.columns.map('_'.join)
            return d.style.format(precision=3)
        # display(prettify(simp_mse_192))
        print(f"SIMPLIFIED {pred_len} MCV")
        display(prettify(sim_mcv_192))
    print("FULL")
    display(pivoted)
    #reformer works best on weather.

# weather

UndefinedVariableError: name 'constraint_level' is not defined

## Pivots per dataset